## Setup (for colab only)

In [ ]:
! git clone https://github.com/theovincent/CPDE.git -b make_ipynb_working

In [ ]:
import os 

os.chdir("/content/CPDE")

! pip install -r requirements.txt

In [ ]:
! git pull

## Imports

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import ruptures as rpt
from SKAB_data.evaluating import evaluating_change_point

%load_ext autoreload
%autoreload 2

from ensemble_methods.aggregations import SCALING_AGGREGATION

SINGLE_COSTS = (
    {'name': 'ar_1', 'cost':'ar', 'params':{'order':1}},
    {'name': 'mahalanobis', 'cost':'mahalanobis', 'params':{}},
    {'name': 'l1', 'cost':'l1', 'params':{}},
    {'name': 'l2', 'cost':'l2', 'params':{}},
    {'name': 'linear', 'cost':'linear', 'params':{}},
    {'name': 'rbf', 'cost': 'rbf', 'params': {}}
)
LIST_COSTS = [dict_cost["cost"] for dict_cost in SINGLE_COSTS]
PARAMS = {"ar": {'order':1}}

DESIRED_ORDER = ["Standart", "LowFP", "LowFN"]

## Load data

In [ ]:
from glob import glob

from sklearn.preprocessing import StandardScaler

# Get the data
files = sorted(glob('TEP_data/*_te.dat'))

columns=[]
for i in range(1, 42):
    columns.append("XMEAS({})".format(i))
for i in range(1, 12):
    columns.append("XMV({})".format(i))

# We do not use the first file because it does not contain a defect
test = {}
for i, j in enumerate(files[1:], start=1):
    test[i] = pd.read_table(j, sep="\s+", names=columns)

# Standardise
for idx_data in test:
    stsc = StandardScaler()
    test[idx_data] = pd.DataFrame(stsc.fit_transform(test[idx_data]), columns=test[idx_data].columns, index=test[idx_data].index)


# Get the labels
INDEX = pd.to_datetime(
        [datetime(2020,1,1)+timedelta(minutes=m) for m in test[1].index*3]
    )
true_cp = pd.Series(data=0, index=INDEX)
# Out of the 48 hours of monitoring, the faults were introduced 8 hours after the beginning
true_cp.iloc[160] = 1
true_cp = [true_cp]*len(test)

## Visualize a signal

In [ ]:
_ = rpt.display(test[1].values[:, 30: 33], [160, 960])

## Window search

In [ ]:
def window_search(cost, params, **kwargs):
    predicted_cp = []
    for idx_data in test:
        algo = rpt.Window(model=cost, 
                          params=params, 
                          width=20,)
        algo.fit(np.array(test[idx_data]))
        my_bkps = algo.predict(n_bkps=1)
        
        single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

table_costs_window = {}
for cost in tqdm(SINGLE_COSTS):
    table_costs_window[cost["name"]] = window_search(**cost)

pd.DataFrame(table_costs_window).T[DESIRED_ORDER]

In [ ]:
def window_search_ensemble_bound():
    predicted_cp = []
    for idx_data in tqdm(test):
        best_nab_sum = - float("inf")
        best_single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)

        for model in SINGLE_COSTS:
            algo = rpt.Window(model=model["cost"], 
                            params=model["params"], 
                            width=20,)
            algo.fit(np.array(test[idx_data]))
            my_bkps = algo.predict(n_bkps=1)
            
            single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
            single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
            
            nab_model = evaluating_change_point([true_cp[0]], [single_predicted_cp], metric='nab', numenta_time='288 min')
            
            if sum(list(nab_model.values())) > best_nab_sum:
                best_nab_sum = sum(list(nab_model.values()))
                best_single_predicted_cp = single_predicted_cp
                
        predicted_cp.append(best_single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

window_ensemble_bound = window_search_ensemble_bound()
pd.DataFrame(window_ensemble_bound, index=["Ensemble bound"])[DESIRED_ORDER]

In [ ]:
from ensemble_methods.window_ensemble import WindowEnsemble

def window_search_ensemble(scale_aggregation):
    predicted_cp = []
    for idx_data in tqdm(test, leave=None, position=1):
        algo = WindowEnsemble(
            width=20,
            models=LIST_COSTS,
            params=PARAMS, 
            scale_aggregation=scale_aggregation,
        )
        single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
        
        algo.fit(np.array(test[idx_data]))
        my_bkps = algo.predict(n_bkps=1)

        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

In [ ]:
table_ensemble_window = {}
for scale_aggregation_name, scale_aggregation in tqdm(SCALING_AGGREGATION.items(), position=0):
    table_ensemble_window[scale_aggregation_name] = window_search_ensemble(scale_aggregation)

pd.DataFrame(table_ensemble_window).T[DESIRED_ORDER]

## Dynamic Programming

In [ ]:
def dynp_search(cost, params, **kwargs):
    predicted_cp = []
    for idx_data in test:
        algo = rpt.Dynp(model=cost, 
                          params=params,
                          )
        algo.fit(np.array(test[idx_data]))
        my_bkps = algo.predict(n_bkps=1)
        
        single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

table_costs_dynp = {}
for cost in tqdm(SINGLE_COSTS):
    table_costs_dynp[cost["name"]] = dynp_search(**cost)

pd.DataFrame(table_costs_dynp).T[DESIRED_ORDER]

In [ ]:
def dynp_search_ensemble_bound():
    predicted_cp = []
    for idx_data in tqdm(test):
        best_nab_sum = - float("inf")
        best_single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)

        for model in SINGLE_COSTS:
            algo = rpt.Dynp(
                    model=model["cost"], 
                    params=model["params"]
                )
            algo.fit(np.array(test[idx_data]))
            my_bkps = algo.predict(n_bkps=1)
            
            single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
            single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
            
            nab_model = evaluating_change_point([true_cp[0]], [single_predicted_cp], metric='nab', numenta_time='288 min')
            
            if sum(list(nab_model.values())) > best_nab_sum:
                best_nab_sum = sum(list(nab_model.values()))
                best_single_predicted_cp = single_predicted_cp
                
        predicted_cp.append(best_single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

dynp_ensemble_bound = dynp_search_ensemble_bound()
pd.DataFrame(dynp_ensemble_bound, index=["Ensemble bound"])[DESIRED_ORDER]

In [ ]:
from ensemble_methods.dynamic_programming_ensemble import DynpEnsemble

def dynamique_programming_ensemble(scale_aggregation):
    predicted_cp = []
    for idx_data in tqdm(test, leave=False, position=1):
        algo = DynpEnsemble(
            models=LIST_COSTS,
            params=PARAMS, 
            scale_aggregation=scale_aggregation,
        )
        single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
        
        algo.fit(np.array(test[idx_data]))
        my_bkps = algo.predict(n_bkps=1)

        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

In [ ]:
table_ensemble_dynp = {}
for scale_aggregation_name, scale_aggregation in tqdm(SCALING_AGGREGATION.items(), position=0):
    table_ensemble_dynp[scale_aggregation_name] = dynamique_programming_ensemble(scale_aggregation)

pd.DataFrame(table_ensemble_dynp).T[DESIRED_ORDER]

## Binary Segmentation

In [ ]:
def binseg_search(cost, params, **kwargs):
    predicted_cp = []
    for idx_data in test:
        algo = rpt.Binseg(model=cost, 
                          params=params,)
        algo.fit(np.array(test[idx_data]))
        my_bkps = algo.predict(n_bkps=1)
        
        single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

table_costs_dynp = {}
for cost in tqdm(SINGLE_COSTS):
    table_costs_dynp[cost["name"]] = binseg_search(**cost)

pd.DataFrame(table_costs_dynp).T[DESIRED_ORDER]

In [ ]:
def binseg_search_ensemble_bound():
    predicted_cp = []
    for idx_data in tqdm(test):
        best_nab_sum = - float("inf")
        best_single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)

        for model in SINGLE_COSTS:
            algo = rpt.Binseg(model=model["cost"], 
                          params=model["params"],)
            algo.fit(np.array(test[idx_data]))
            my_bkps = algo.predict(n_bkps=1)
            
            single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
            single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
            
            nab_model = evaluating_change_point([true_cp[0]], [single_predicted_cp], metric='nab', numenta_time='288 min')
            
            if sum(list(nab_model.values())) > best_nab_sum:
                best_nab_sum = sum(list(nab_model.values()))
                best_single_predicted_cp = single_predicted_cp
                
        predicted_cp.append(best_single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

binseg_ensemble_bound = binseg_search_ensemble_bound()
pd.DataFrame(binseg_ensemble_bound, index=["Ensemble bound"])[DESIRED_ORDER]

In [ ]:
from ensemble_methods.binary_segmentation_ensemble import BinsegEnsemble

def binary_segmentation_ensemble(scale_aggregation):
    predicted_cp = []
    for idx_data in tqdm(test, leave=False, position=1):
        algo = BinsegEnsemble(
            models=LIST_COSTS,
            params=PARAMS, 
            scale_aggregation=scale_aggregation,
        )
        single_predicted_cp = pd.Series(data=0, index=true_cp[0].index)
        
        algo.fit(np.array(test[idx_data]))
        my_bkps = algo.predict(n_bkps=1)

        single_predicted_cp[single_predicted_cp.index[my_bkps[:-1]]] = 1
        predicted_cp.append(single_predicted_cp)

    nab = evaluating_change_point(true_cp, predicted_cp, metric='nab', numenta_time='288 min')
    return nab

In [ ]:
table_ensemble_dynp = {}
for scale_aggregation_name, scale_aggregation in tqdm(SCALING_AGGREGATION.items(), position=0):
    table_ensemble_dynp[scale_aggregation_name] = binary_segmentation_ensemble(scale_aggregation)

pd.DataFrame(table_ensemble_dynp).T[DESIRED_ORDER]